<a href="https://colab.research.google.com/github/Untick/IrisID_gr2/blob/Alexey-Tatarinov-folder/Alexey%20Tatarinov/Iris_Autokeras_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Импорт библиотек и модулей**

In [1]:
import gdown

import os

! pip install autokeras
import autokeras as ak

! pip install Augmentor
import Augmentor

import zipfile

import numpy as np

import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model

from tensorflow.keras.layers import (Dense, Conv2D, MaxPooling2D, Flatten, 
                                     Dropout, BatchNormalization, Rescaling,
                                     GlobalAveragePooling2D, RandomFlip, Input,
                                     RandomRotation, RandomZoom, RandomContrast)

from tensorflow.keras.applications import VGG19, ResNet50, Xception

# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import (ImageDataGenerator, 
                                                 DirectoryIterator)

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import OneHotEncoder

from PIL import Image, ImageEnhance

import matplotlib.pyplot as plt

%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 111.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Подготовка датасета**

## **Задание гиперпараметров**

In [2]:
TRAIN_PATH          = '/content/drive/MyDrive/iris_ds'
TRAIN_PATH_AUG      = '/content/drive/MyDrive/Iris'

VAL_SPLIT           = 0.2

IMG_WIDTH_1         = 160
IMG_HEIGHT_1        = 120
IMG_WIDTH_2         = 80
IMG_HEIGHT_2        = 60
IMG_CHANNELS        = 3

ROTATION_RANGE      = 10
WIDTH_SHIFT_RANGE   = 0.1
HEIGHT_SHIFT_RANGE  = 0.1
ZOOM_RANGE          = 0.1
BRIGHTNESS_RANGE    = (0.5, 1.3)
HORIZONTAL_FLIP     = False

EPOCHS              = 20
BATCH_SIZE          = 24
OPTIMIZER_LEGACY    = tf.keras.optimizers.legacy.Adam(0.0001)
OPTIMIZER           = tf.keras.optimizers.Adam(0.0001)

## **Загрузка датасета**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Аугментация**

In [ ]:
for folder in os.listdir(TRAIN_PATH):
    if os.path.isdir(os.path.join(TRAIN_PATH, folder)):
        # Создаем генератор аугментации для каждой папки
        p = Augmentor.Pipeline(os.path.join(TRAIN_PATH, folder), output_directory=os.path.join(f'/content/drive/MyDrive/Iris/{folder}'))
        # Добавляем операции аугментации
        # Вращение изображения
        p.rotate(probability=0.2, max_left_rotation=10, max_right_rotation=10)
        # увеличение
        #p.zoom(probability=0.2, min_factor=1.1, max_factor=1.2)
        # Обрезание изображения
        p.crop_random(probability=0.1, percentage_area=0.9)
        # Изменение яркости
        p.random_brightness(probability=0.5, min_factor=0.5, max_factor=1.3)
        # Изменение контрасности
        p.random_contrast(probability=0.5, min_factor=0.5, max_factor=1.3)
        # Применяем аугментацию
        p.sample(50)
print('Аугментация завершена')

## **Создание базы**

In [4]:
CLASS_LIST = sorted(os.listdir(TRAIN_PATH_AUG))
CLASS_COUNT = len(CLASS_LIST)

print(f'Количество классов: {CLASS_COUNT}, метки классов: {CLASS_LIST}')

Количество классов: 64, метки классов: ['client_1', 'client_10', 'client_11', 'client_12', 'client_13', 'client_14', 'client_15', 'client_16', 'client_17', 'client_18', 'client_19', 'client_2', 'client_20', 'client_21', 'client_22', 'client_23', 'client_24', 'client_25', 'client_26', 'client_27', 'client_28', 'client_29', 'client_3', 'client_30', 'client_31', 'client_32', 'client_33', 'client_34', 'client_35', 'client_36', 'client_37', 'client_38', 'client_39', 'client_4', 'client_40', 'client_41', 'client_42', 'client_43', 'client_44', 'client_45', 'client_46', 'client_47', 'client_48', 'client_49', 'client_5', 'client_50', 'client_51', 'client_52', 'client_53', 'client_54', 'client_55', 'client_56', 'client_57', 'client_58', 'client_59', 'client_6', 'client_60', 'client_61', 'client_62', 'client_63', 'client_64', 'client_7', 'client_8', 'client_9']


In [5]:
data_files = []
data_labels = []

for class_label in range(CLASS_COUNT):    # Для всех классов по порядку номеров (их меток)
    class_name = CLASS_LIST[class_label]  # Выборка имени класса из списка имен
    class_path = f'{TRAIN_PATH_AUG}/{class_name}'  # Формирование полного пути к папке с изображениями класса
    class_files = os.listdir(class_path)  # Получение списка имен файлов с изображениями текущего класса
    data_files += [f'{class_path}/{file_name}' for file_name in class_files]
    data_labels += [class_label] * len(class_files)
    print(f'Размер класса {class_name} составляет {len(class_files)} снимков')

print('Общий размер базы для обучения:', len(data_labels))

Размер класса client_1 составляет 50 снимков
Размер класса client_10 составляет 50 снимков
Размер класса client_11 составляет 50 снимков
Размер класса client_12 составляет 50 снимков
Размер класса client_13 составляет 50 снимков
Размер класса client_14 составляет 50 снимков
Размер класса client_15 составляет 50 снимков
Размер класса client_16 составляет 50 снимков
Размер класса client_17 составляет 50 снимков
Размер класса client_18 составляет 50 снимков
Размер класса client_19 составляет 50 снимков
Размер класса client_2 составляет 50 снимков
Размер класса client_20 составляет 50 снимков
Размер класса client_21 составляет 50 снимков
Размер класса client_22 составляет 50 снимков
Размер класса client_23 составляет 50 снимков
Размер класса client_24 составляет 50 снимков
Размер класса client_25 составляет 50 снимков
Размер класса client_26 составляет 50 снимков
Размер класса client_27 составляет 50 снимков
Размер класса client_28 составляет 50 снимков
Размер класса client_29 составляет 5

In [7]:
# data_images_1 = []

# for file_name in data_files:
#     img = Image.open(file_name).resize((IMG_WIDTH_1, IMG_HEIGHT_1)) 
#     img_np = np.array(img)
#     data_images_1.append(img_np)

# x_data_big = np.array(data_images_1)
# y_data = np.array(data_labels)

# print(f'В массив собрано {len(data_images_1)} фотографий следующей формы: {img_np.shape}')
# print(f'Общий массив данных изображений следующей формы: {x_data_big.shape}')
# print(f'Общий массив меток классов следующей формы: {y_data.shape}')

data_images = []

for file_name in data_files:
    img = Image.open(file_name).resize((IMG_WIDTH_2, IMG_HEIGHT_2)) 
    img_np = np.array(img)
    data_images.append(img_np)

x_data_small = np.array(data_images)
y_data = np.array(data_labels)

print(f'В массив собрано {len(data_images)} фотографий следующей формы: {img_np.shape}')
print(f'Общий массив данных изображений следующей формы: {x_data_small.shape}')
print(f'Общий массив меток классов следующей формы: {y_data.shape}')

В массив собрано 3200 фотографий следующей формы: (60, 80, 3)
Общий массив данных изображений следующей формы: (3200, 60, 80, 3)
Общий массив меток классов следующей формы: (3200,)


## **Деление на выборки**

In [8]:
# Разделение выборки на обучающую, проверочную и тестовую
# x_ds_big, x_val_big, y_ds_big, y_val_big = train_test_split(x_data_big, y_data, test_size=0.2, random_state=42)
# x_train_big, x_test_big, y_train_big, y_test_big = train_test_split(x_ds_big, y_ds_big, test_size=0.1, random_state=42)

x_ds_small, x_val_small, y_ds_small, y_val_small = train_test_split(x_data_small, y_data, test_size=0.2, random_state=42)
x_train_small, x_test_small, y_train_small, y_test_small = train_test_split(x_ds_small, y_ds_small, test_size=0.1, random_state=42)

In [9]:
y_train_small = to_categorical(y_train_small)
# y_train_big = to_categorical(y_train_big)

y_val_small = to_categorical(y_val_small)
# y_val_big = to_categorical(y_val_big)

y_test_small = to_categorical(y_test_small)
# y_test_big = to_categorical(y_test_big)

In [10]:
# print('Размерности датасетов с изображениями 160 на 120')
# print('Тренировочный датасет:')
# print(x_train_big.shape)
# print(y_train_big.shape)
# print('Проверочный датасет:')
# print(x_val_big.shape)
# print(y_val_big.shape)
# print('Тестовый датасет:')
# print(x_test_big.shape)
# print(y_test_big.shape)

print('\nРазмерности датасетов с изображениями 80 на 60')
print('Тренировочный датасет:')
print(x_train_small.shape)
print(y_train_small.shape)
print('Проверочный датасет:')
print(x_val_small.shape)
print(y_val_small.shape)
print('Тестовый датасет:')
print(x_test_small.shape)
print(y_test_small.shape)


Размерности датасетов с изображениями 80 на 60
Тренировочный датасет:
(2304, 60, 80, 3)
(2304, 64)
Проверочный датасет:
(640, 60, 80, 3)
(640, 64)
Тестовый датасет:
(256, 60, 80, 3)
(256, 64)


# **Оптимизация обучения модели**

In [ ]:
# AUTOTUNE = tf.data.AUTOTUNE
# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [11]:
checkpoint = ModelCheckpoint(filepath="my_model.hdf5", monitor="val_loss", verbose=0, save_best_only=True)

# **Работа с моделью**

In [11]:
clf = ak.ImageClassifier(
    max_trials=2,
    objective='val_accuracy',
    directory='automl_irisID_1',
    overwrite=True)

clf.fit(x_train_small, y_train_small, epochs=10, validation_data=(x_val_small, y_val_small))

Trial 2 Complete [00h 06m 06s]
val_accuracy: 0.5234375

Best val_accuracy So Far: 0.981249988079071
Total elapsed time: 00h 06m 31s
Epoch 1/10
72/72 [==============================] - 2s 17ms/step - loss: 2.7656 - accuracy: 0.3411 - val_loss: 0.9485 - val_accuracy: 0.7531
Epoch 2/10
72/72 [==============================] - 1s 16ms/step - loss: 0.4621 - accuracy: 0.8720 - val_loss: 0.2634 - val_accuracy: 0.9281
Epoch 3/10
72/72 [==============================] - 1s 16ms/step - loss: 0.1449 - accuracy: 0.9596 - val_loss: 0.1397 - val_accuracy: 0.9641
Epoch 4/10
72/72 [==============================] - 1s 15ms/step - loss: 0.0420 - accuracy: 0.9909 - val_loss: 0.1379 - val_accuracy: 0.9594
Epoch 5/10
72/72 [==============================] - 1s 14ms/step - loss: 0.0313 - accuracy: 0.9922 - val_loss: 0.1053 - val_accuracy: 0.9703
Epoch 6/10
72/72 [==============================] - 1s 14ms/step - loss: 0.0261 - accuracy: 0.9931 - val_loss: 0.1617 - val_accuracy: 0.9578
Epoch 7/10
72/72 [====

In [12]:
# clf.save('model_autokeras_iris_v1', save_format="h5")
best_model = clf.export_model()
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 60, 80, 3)]       0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 60, 80, 3)        0         
 t32)                                                            
                                                                 
 normalization (Normalizatio  (None, 60, 80, 3)        7         
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 58, 78, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 76, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 38, 64)       0     

# **Проверка модели**

## **Визуализация результатов работы модели**

In [15]:
# Визуализация точности на обучающей выборке
plt.plot(clf.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')

# Визуализация точности на проверочной выборке
plt.plot(clf.history['val_accuracy'], 
         label='Доля верных ответов на проверочном наборе')

# Отрисовка подписей осей
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')

# Отрисовка легенды
plt.legend()

# Вывод графика
plt.show()

TypeError: ignored

In [ ]:
# Визуализация точности на обучающей выборке
plt.plot(history_small.history['loss'], 
         label='Ошибка на обучающем наборе')

# Визуализация точности на проверочной выборке
plt.plot(history_small.history['val_loss'], 
         label='Ошибка на проверочном наборе')

# Отрисовка подписей осей
plt.xlabel('Эпоха обучения')
plt.ylabel('Ошибка')

plt.legend()

plt.show()

## **Проверка evaluate**

In [17]:
best_model = clf.export_model()
best_model.summary()

8/8 [==============================] - 0s 7ms/step - loss: 0.0657 - accuracy: 0.9883
Процент верных ответов на тестовых данных размером 80 на 60: 98.83 %


In [ ]:
scores_small = best_model.evaluate(x_test_small, 
                                   y_test_small, 
                                   verbose=1
                                   )

print('Процент верных ответов на тестовых данных размером 80 на 60:', round(scores_small[1],4) * 100, '%')

## **Проверка predict**

In [18]:
x = x_test_small[10]
print(x.shape)

x = np.expand_dims(x, axis=0)
print(x.shape)

prediction = clf.predict(x) 
print(prediction)

pred = np.argmax(prediction)
print(f'Распознана цифра: {pred}')
print(f'Исходная цифра: {np.argmax(y_test_small[10])}')

(60, 80, 3)
(1, 60, 80, 3)
1/1 [==============================] - 0s 7ms/step
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Распознана цифра: 5
Исходная цифра: 5


# **Изменение версии tensorflow**

In [2]:
pip show tensorflow

Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, jax, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl


In [3]:
pip install --upgrade tensorflow==<2.9.1>

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `pip install --upgrade tensorflow==<2.9.1>'
